# Machine Learning Tutorial: A Comprehensive Guide

Welcome to this comprehensive machine learning tutorial! In this notebook, we'll explore the fundamentals of machine learning using Python and popular libraries like scikit-learn, pandas, and matplotlib.

## What You'll Learn:
- Data preprocessing and cleaning techniques
- Exploratory data analysis and visualization
- Supervised learning algorithms (Linear Regression, Decision Trees, Random Forest)
- Model evaluation and comparison
- Cross-validation and hyperparameter tuning
- Feature importance analysis
- Making predictions on new data

Let's dive into the exciting world of machine learning!

## 1. Import Required Libraries

First, we'll import all the essential libraries we'll need for this tutorial. Each library serves a specific purpose in our machine learning workflow.

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Machine learning libraries
from sklearn.datasets import load_boston, load_iris
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Configure plotting style
plt.style.use('default')
sns.set_palette("husl")

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")
print("Python packages versions:")
print(f"pandas: {pd.__version__}")
print(f"numpy: {np.__version__}")
print(f"matplotlib: {plt.matplotlib.__version__}")
print(f"seaborn: {sns.__version__}")

## 2. Load and Explore Dataset

We'll use the Boston Housing dataset, which is perfect for regression tasks. This dataset contains information about housing values in Boston suburbs and various features that might influence housing prices.

In [ ]:
# Load the Boston Housing dataset
boston = load_boston()

# Create a DataFrame for easier manipulation
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['PRICE'] = boston.target  # Add the target variable

print("Dataset loaded successfully!")
print(f"Dataset shape: {df.shape}")
print(f"Number of features: {len(boston.feature_names)}")
print(f"Target variable: House prices in $1000s")

print("\nFirst 5 rows of the dataset:")
print(df.head())

print("\nDataset information:")
print(df.info())

print("\nBasic statistics:")
print(df.describe())

### Feature Descriptions:
- **CRIM**: Crime rate per capita
- **ZN**: Proportion of residential land zoned for lots over 25,000 sq.ft
- **INDUS**: Proportion of non-retail business acres
- **CHAS**: Charles River dummy variable (1 if tract bounds river; 0 otherwise)
- **NOX**: Nitric oxides concentration (parts per 10 million)
- **RM**: Average number of rooms per dwelling
- **AGE**: Proportion of owner-occupied units built prior to 1940
- **DIS**: Weighted distances to employment centers
- **RAD**: Index of accessibility to radial highways
- **TAX**: Property tax rate per $10,000
- **PTRATIO**: Pupil-teacher ratio by town
- **B**: Proportion of blacks by town
- **LSTAT**: % lower status of the population
- **PRICE**: Median home value in $1000s (our target variable)

## 3. Data Preprocessing and Cleaning

Before building machine learning models, we need to ensure our data is clean and properly formatted.

In [ ]:
# Check for missing values
print("Missing values in each column:")
print(df.isnull().sum())

# Check for duplicate rows
print(f"\nNumber of duplicate rows: {df.duplicated().sum()}")

# Check data types
print(f"\nData types:")
print(df.dtypes)

# Look for outliers using the IQR method
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return len(outliers)

print(f"\nOutlier detection (using IQR method):")
for column in df.columns:
    outlier_count = detect_outliers(df, column)
    print(f"{column}: {outlier_count} outliers")

print(f"\nDataset is clean! No missing values or duplicates found.")
print(f"Some outliers detected, but this is normal for real-world data.")

## 4. Exploratory Data Analysis

Let's visualize our data to understand patterns, distributions, and relationships between variables.

In [ ]:
# Set up the plotting area
plt.figure(figsize=(15, 10))

# Distribution of the target variable (house prices)
plt.subplot(2, 3, 1)
plt.hist(df['PRICE'], bins=30, edgecolor='black', alpha=0.7)
plt.title('Distribution of House Prices')
plt.xlabel('Price ($1000s)')
plt.ylabel('Frequency')

# Correlation heatmap
plt.subplot(2, 3, 2)
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0)
plt.title('Feature Correlation Heatmap')

# Scatter plot: RM (rooms) vs Price
plt.subplot(2, 3, 3)
plt.scatter(df['RM'], df['PRICE'], alpha=0.6)
plt.title('Rooms vs House Price')
plt.xlabel('Average Number of Rooms')
plt.ylabel('Price ($1000s)')

# Scatter plot: LSTAT vs Price
plt.subplot(2, 3, 4)
plt.scatter(df['LSTAT'], df['PRICE'], alpha=0.6, color='orange')
plt.title('Lower Status % vs House Price')
plt.xlabel('% Lower Status Population')
plt.ylabel('Price ($1000s)')

# Box plot: CHAS (Charles River) vs Price
plt.subplot(2, 3, 5)
df.boxplot(column='PRICE', by='CHAS', ax=plt.gca())
plt.title('House Prices by Charles River Location')
plt.xlabel('Charles River (0=No, 1=Yes)')
plt.ylabel('Price ($1000s)')

# Crime rate vs Price
plt.subplot(2, 3, 6)
plt.scatter(df['CRIM'], df['PRICE'], alpha=0.6, color='red')
plt.title('Crime Rate vs House Price')
plt.xlabel('Crime Rate per Capita')
plt.ylabel('Price ($1000s)')

plt.tight_layout()
plt.show()

# Print top correlations with price
price_corr = df.corr()['PRICE'].abs().sort_values(ascending=False)
print("Features most correlated with house price:")
print(price_corr[1:6])  # Exclude price itself

## 5. Feature Selection and Engineering

We'll prepare our features for machine learning by selecting the most relevant ones and applying necessary transformations.

In [ ]:
# Separate features and target variable
X = df.drop('PRICE', axis=1)  # Features
y = df['PRICE']               # Target variable

print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

# Let's create some new features that might be useful
# Feature engineering: Create new meaningful features

# 1. Rooms per capita (rooms divided by population density)
X['ROOMS_PER_CAPITA'] = X['RM'] / (X['CRIM'] + 1)  # Adding 1 to avoid division by zero

# 2. Tax per room (property tax divided by number of rooms)
X['TAX_PER_ROOM'] = X['TAX'] / X['RM']

# 3. Distance-accessibility interaction
X['DIS_RAD_INTERACTION'] = X['DIS'] * X['RAD']

print("\nNew features created:")
print("- ROOMS_PER_CAPITA: Rooms relative to crime rate")
print("- TAX_PER_ROOM: Property tax per room")
print("- DIS_RAD_INTERACTION: Distance-accessibility interaction")

# Select features based on correlation with target
feature_importance = abs(X.corrwith(y)).sort_values(ascending=False)
print("\nFeature importance (correlation with price):")
print(feature_importance.head(10))

# Select top features for modeling
top_features = feature_importance.head(8).index.tolist()
X_selected = X[top_features]

print(f"\nSelected {len(top_features)} most important features:")
print(top_features)

## 6. Split Data into Training and Testing Sets

We'll split our data to train our models on one portion and test them on unseen data to evaluate their performance.

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42, stratify=None
)

print("Data split completed!")
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")
print(f"Number of features: {X_train.shape[1]}")

# Scale the features for algorithms that are sensitive to feature scales
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nFeature scaling completed!")
print(f"Original feature ranges:")
print(f"Min values: {X_train.min()}")
print(f"Max values: {X_train.max()}")
print(f"\nScaled feature ranges:")
print(f"Min values: {X_train_scaled.min(axis=0)}")
print(f"Max values: {X_train_scaled.max(axis=0)}")

# Convert scaled arrays back to DataFrames for easier handling
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

## 7. Train a Linear Regression Model

Linear regression is one of the simplest and most interpretable machine learning algorithms. It assumes a linear relationship between features and the target variable.

In [ ]:
# Create and train the Linear Regression model
lr_model = LinearRegression()

# Train the model
lr_model.fit(X_train_scaled, y_train)

# Make predictions
lr_predictions = lr_model.predict(X_test_scaled)

# Calculate performance metrics
lr_mse = mean_squared_error(y_test, lr_predictions)
lr_rmse = np.sqrt(lr_mse)
lr_r2 = r2_score(y_test, lr_predictions)

print("Linear Regression Results:")
print(f"Mean Squared Error (MSE): {lr_mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {lr_rmse:.2f}")
print(f"R² Score: {lr_r2:.3f}")

# Display feature coefficients
feature_coefficients = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': lr_model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print(f"\nFeature Coefficients (sorted by absolute value):")
print(feature_coefficients)

# Visualize predictions vs actual values
plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1)
plt.scatter(y_test, lr_predictions, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices')
plt.title('Linear Regression: Actual vs Predicted')

plt.subplot(1, 2, 2)
residuals = y_test - lr_predictions
plt.scatter(lr_predictions, residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Prices')
plt.ylabel('Residuals')
plt.title('Residual Plot')

plt.tight_layout()
plt.show()

## 8. Train a Decision Tree Model

Decision trees are intuitive models that make decisions by asking a series of questions about the features. They're highly interpretable and can capture non-linear relationships.

In [ ]:
# Create and train the Decision Tree model
# Note: Decision trees don't require feature scaling, so we'll use the original features
dt_model = DecisionTreeRegressor(max_depth=6, min_samples_split=10, random_state=42)

# Train the model
dt_model.fit(X_train, y_train)

# Make predictions
dt_predictions = dt_model.predict(X_test)

# Calculate performance metrics
dt_mse = mean_squared_error(y_test, dt_predictions)
dt_rmse = np.sqrt(dt_mse)
dt_r2 = r2_score(y_test, dt_predictions)

print("Decision Tree Results:")
print(f"Mean Squared Error (MSE): {dt_mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {dt_rmse:.2f}")
print(f"R² Score: {dt_r2:.3f}")

# Feature importance from the decision tree
dt_feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': dt_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\nFeature Importance (Decision Tree):")
print(dt_feature_importance)

# Visualize the results
plt.figure(figsize=(15, 5))

# Predictions vs Actual
plt.subplot(1, 3, 1)
plt.scatter(y_test, dt_predictions, alpha=0.6, color='green')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices')
plt.title('Decision Tree: Actual vs Predicted')

# Feature importance plot
plt.subplot(1, 3, 2)
plt.barh(dt_feature_importance['Feature'], dt_feature_importance['Importance'])
plt.xlabel('Importance')
plt.title('Feature Importance (Decision Tree)')

# Residuals plot
plt.subplot(1, 3, 3)
dt_residuals = y_test - dt_predictions
plt.scatter(dt_predictions, dt_residuals, alpha=0.6, color='green')
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Prices')
plt.ylabel('Residuals')
plt.title('Residual Plot (Decision Tree)')

plt.tight_layout()
plt.show()

# Print some information about the tree
print(f"\nDecision Tree Information:")
print(f"Tree depth: {dt_model.get_depth()}")
print(f"Number of leaves: {dt_model.get_n_leaves()}")

## 9. Train a Random Forest Model

Random Forest is an ensemble method that combines multiple decision trees to create a more robust and accurate model. It reduces overfitting and provides better generalization.

In [ ]:
# Create and train the Random Forest model
rf_model = RandomForestRegressor(
    n_estimators=100,        # Number of trees in the forest
    max_depth=8,             # Maximum depth of trees
    min_samples_split=5,     # Minimum samples required to split a node
    min_samples_leaf=2,      # Minimum samples required at a leaf node
    random_state=42          # For reproducibility
)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
rf_predictions = rf_model.predict(X_test)

# Calculate performance metrics
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_rmse = np.sqrt(rf_mse)
rf_r2 = r2_score(y_test, rf_predictions)

print("Random Forest Results:")
print(f"Mean Squared Error (MSE): {rf_mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rf_rmse:.2f}")
print(f"R² Score: {rf_r2:.3f}")

# Feature importance from Random Forest
rf_feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\nFeature Importance (Random Forest):")
print(rf_feature_importance)

# Visualize the results
plt.figure(figsize=(15, 5))

# Predictions vs Actual
plt.subplot(1, 3, 1)
plt.scatter(y_test, rf_predictions, alpha=0.6, color='purple')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices')
plt.title('Random Forest: Actual vs Predicted')

# Feature importance comparison
plt.subplot(1, 3, 2)
plt.barh(rf_feature_importance['Feature'], rf_feature_importance['Importance'], color='purple', alpha=0.7)
plt.xlabel('Importance')
plt.title('Feature Importance (Random Forest)')

# Residuals plot
plt.subplot(1, 3, 3)
rf_residuals = y_test - rf_predictions
plt.scatter(rf_predictions, rf_residuals, alpha=0.6, color='purple')
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Prices')
plt.ylabel('Residuals')
plt.title('Residual Plot (Random Forest)')

plt.tight_layout()
plt.show()

print(f"\nRandom Forest Information:")
print(f"Number of trees: {rf_model.n_estimators}")
print(f"Out-of-bag score: {rf_model.oob_score_:.3f}" if hasattr(rf_model, 'oob_score_') else "OOB score not available")

## 10. Model Evaluation and Comparison

Let's compare all three models to see which one performs best on our dataset.

In [ ]:
# Create a comparison DataFrame
model_comparison = pd.DataFrame({
    'Model': ['Linear Regression', 'Decision Tree', 'Random Forest'],
    'MSE': [lr_mse, dt_mse, rf_mse],
    'RMSE': [lr_rmse, dt_rmse, rf_rmse],
    'R² Score': [lr_r2, dt_r2, rf_r2]
})

print("Model Performance Comparison:")
print(model_comparison)

# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# MSE Comparison
axes[0, 0].bar(model_comparison['Model'], model_comparison['MSE'], 
               color=['blue', 'green', 'purple'], alpha=0.7)
axes[0, 0].set_title('Mean Squared Error Comparison')
axes[0, 0].set_ylabel('MSE')
axes[0, 0].tick_params(axis='x', rotation=45)

# R² Score Comparison
axes[0, 1].bar(model_comparison['Model'], model_comparison['R² Score'], 
               color=['blue', 'green', 'purple'], alpha=0.7)
axes[0, 1].set_title('R² Score Comparison')
axes[0, 1].set_ylabel('R² Score')
axes[0, 1].tick_params(axis='x', rotation=45)

# Prediction comparison plot
axes[1, 0].scatter(y_test, lr_predictions, alpha=0.5, label='Linear Regression', color='blue')
axes[1, 0].scatter(y_test, dt_predictions, alpha=0.5, label='Decision Tree', color='green')
axes[1, 0].scatter(y_test, rf_predictions, alpha=0.5, label='Random Forest', color='purple')
axes[1, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 0].set_xlabel('Actual Prices')
axes[1, 0].set_ylabel('Predicted Prices')
axes[1, 0].set_title('All Models: Actual vs Predicted')
axes[1, 0].legend()

# Residuals comparison
axes[1, 1].scatter(lr_predictions, lr_residuals, alpha=0.5, label='Linear Regression', color='blue')
axes[1, 1].scatter(dt_predictions, dt_residuals, alpha=0.5, label='Decision Tree', color='green')
axes[1, 1].scatter(rf_predictions, rf_residuals, alpha=0.5, label='Random Forest', color='purple')
axes[1, 1].axhline(y=0, color='r', linestyle='--')
axes[1, 1].set_xlabel('Predicted Prices')
axes[1, 1].set_ylabel('Residuals')
axes[1, 1].set_title('Residuals Comparison')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Find the best model
best_model_idx = model_comparison['R² Score'].idxmax()
best_model = model_comparison.loc[best_model_idx, 'Model']
best_r2 = model_comparison.loc[best_model_idx, 'R² Score']

print(f"\n🏆 Best performing model: {best_model}")
print(f"   R² Score: {best_r2:.3f}")
print(f"   This means the model explains {best_r2*100:.1f}% of the variance in house prices.")

## 11. Cross-Validation

Cross-validation helps us get a more robust estimate of model performance by training and testing on different subsets of the data.

In [ ]:
# Perform 5-fold cross-validation for all models
from sklearn.model_selection import cross_val_score

# Prepare models for cross-validation
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(max_depth=6, min_samples_split=10, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=8, random_state=42)
}

# Perform cross-validation
cv_results = {}
for name, model in models.items():
    if name == 'Linear Regression':
        # Use scaled features for Linear Regression
        cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
    else:
        # Use original features for tree-based models
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    
    cv_results[name] = {
        'scores': cv_scores,
        'mean': cv_scores.mean(),
        'std': cv_scores.std()
    }

# Display cross-validation results
print("5-Fold Cross-Validation Results (R² Score):")
print("=" * 50)

cv_summary = []
for name, results in cv_results.items():
    print(f"{name}:")
    print(f"  Mean R² Score: {results['mean']:.3f} (±{results['std']:.3f})")
    print(f"  Individual folds: {[f'{score:.3f}' for score in results['scores']]}")
    print()
    
    cv_summary.append({
        'Model': name,
        'Mean R²': results['mean'],
        'Std R²': results['std']
    })

# Create DataFrame for visualization
cv_df = pd.DataFrame(cv_summary)

# Visualize cross-validation results
plt.figure(figsize=(12, 5))

# Bar plot of mean scores with error bars
plt.subplot(1, 2, 1)
plt.bar(cv_df['Model'], cv_df['Mean R²'], 
        yerr=cv_df['Std R²'], capsize=5, 
        color=['blue', 'green', 'purple'], alpha=0.7)
plt.title('Cross-Validation Results (Mean ± Std)')
plt.ylabel('R² Score')
plt.xticks(rotation=45)

# Box plot of all CV scores
plt.subplot(1, 2, 2)
cv_scores_list = [cv_results[name]['scores'] for name in models.keys()]
plt.boxplot(cv_scores_list, labels=models.keys())
plt.title('Distribution of CV Scores')
plt.ylabel('R² Score')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# Find the best model based on cross-validation
best_cv_model = cv_df.loc[cv_df['Mean R²'].idxmax(), 'Model']
best_cv_score = cv_df.loc[cv_df['Mean R²'].idxmax(), 'Mean R²']

print(f"🏆 Best model based on cross-validation: {best_cv_model}")
print(f"   Mean R² Score: {best_cv_score:.3f}")
print(f"   This provides a more robust estimate of model performance!")

## 12. Hyperparameter Tuning

Let's optimize our best performing model by finding the optimal hyperparameters using Grid Search.

In [ ]:
# Let's tune the Random Forest model (assuming it's our best performer)
from sklearn.model_selection import GridSearchCV

# Define hyperparameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],           # Number of trees
    'max_depth': [6, 8, 10, None],           # Maximum depth of trees
    'min_samples_split': [2, 5, 10],         # Minimum samples to split
    'min_samples_leaf': [1, 2, 4],           # Minimum samples at leaf
    'max_features': ['auto', 'sqrt', 'log2']  # Number of features to consider
}

print("Starting hyperparameter tuning for Random Forest...")
print(f"Total combinations to test: {len(param_grid['n_estimators']) * len(param_grid['max_depth']) * len(param_grid['min_samples_split']) * len(param_grid['min_samples_leaf']) * len(param_grid['max_features'])}")

# Create GridSearchCV object
grid_search = GridSearchCV(
    RandomForestRegressor(random_state=42),
    param_grid,
    cv=5,                    # 5-fold cross-validation
    scoring='r2',            # R² score as the metric
    n_jobs=-1,              # Use all available cores
    verbose=1               # Show progress
)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Get the best model
best_rf_model = grid_search.best_estimator_

print("Hyperparameter tuning completed!")
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.3f}")

# Make predictions with the tuned model
best_rf_predictions = best_rf_model.predict(X_test)

# Calculate performance metrics for the tuned model
best_rf_mse = mean_squared_error(y_test, best_rf_predictions)
best_rf_rmse = np.sqrt(best_rf_mse)
best_rf_r2 = r2_score(y_test, best_rf_predictions)

print(f"\nTuned Random Forest Results:")
print(f"Mean Squared Error (MSE): {best_rf_mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {best_rf_rmse:.2f}")
print(f"R² Score: {best_rf_r2:.3f}")

# Compare with original Random Forest
print(f"\nImprovement over original Random Forest:")
print(f"MSE improvement: {rf_mse - best_rf_mse:.2f}")
print(f"R² improvement: {best_rf_r2 - rf_r2:.3f}")

# Visualize the improvement
plt.figure(figsize=(15, 5))

# Comparison of R² scores
models_comparison = ['Original RF', 'Tuned RF']
r2_scores = [rf_r2, best_rf_r2]

plt.subplot(1, 3, 1)
plt.bar(models_comparison, r2_scores, color=['purple', 'darkblue'], alpha=0.7)
plt.title('Model Performance: Before vs After Tuning')
plt.ylabel('R² Score')

# Predictions comparison
plt.subplot(1, 3, 2)
plt.scatter(y_test, rf_predictions, alpha=0.5, label='Original RF', color='purple')
plt.scatter(y_test, best_rf_predictions, alpha=0.5, label='Tuned RF', color='darkblue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices')
plt.title('Predictions: Original vs Tuned RF')
plt.legend()

# Feature importance for tuned model
best_rf_feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': best_rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.subplot(1, 3, 3)
plt.barh(best_rf_feature_importance['Feature'], best_rf_feature_importance['Importance'], 
         color='darkblue', alpha=0.7)
plt.xlabel('Importance')
plt.title('Feature Importance (Tuned RF)')

plt.tight_layout()
plt.show()

print(f"\nTop 5 most important features (tuned model):")
print(best_rf_feature_importance.head())

## 13. Feature Importance Analysis

Let's analyze which features are most important for predicting house prices and understand how our model makes decisions.

In [ ]:
# Comprehensive feature importance analysis
print("Feature Importance Analysis")
print("=" * 50)

# 1. Feature importance from our best model
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': best_rf_model.feature_importances_,
    'Importance_Percentage': best_rf_model.feature_importances_ * 100
}).sort_values('Importance', ascending=False)

print("Feature Importance Rankings:")
for idx, row in feature_importance_df.iterrows():
    print(f"{row.name + 1:2d}. {row['Feature']:20s} - {row['Importance_Percentage']:5.1f}%")

# 2. Correlation with target variable
target_correlation = abs(X_train.corrwith(y_train)).sort_values(ascending=False)

print(f"\nCorrelation with house prices:")
for feature in target_correlation.index[:5]:
    corr = X_train[feature].corr(y_train)
    print(f"{feature:20s} - {corr:6.3f}")

# 3. Visualize feature importance
plt.figure(figsize=(15, 10))

# Main feature importance plot
plt.subplot(2, 2, 1)
colors = plt.cm.viridis(np.linspace(0, 1, len(feature_importance_df)))
plt.barh(range(len(feature_importance_df)), feature_importance_df['Importance'], color=colors)
plt.yticks(range(len(feature_importance_df)), feature_importance_df['Feature'])
plt.xlabel('Importance')
plt.title('Feature Importance (Random Forest)')
plt.gca().invert_yaxis()

# Top 5 features pie chart
plt.subplot(2, 2, 2)
top_5_features = feature_importance_df.head(5)
plt.pie(top_5_features['Importance'], labels=top_5_features['Feature'], autopct='%1.1f%%')
plt.title('Top 5 Features Distribution')

# Feature importance vs correlation
plt.subplot(2, 2, 3)
correlation_values = [abs(X_train[feature].corr(y_train)) for feature in feature_importance_df['Feature']]
plt.scatter(feature_importance_df['Importance'], correlation_values, alpha=0.7)
plt.xlabel('Random Forest Importance')
plt.ylabel('Correlation with Target')
plt.title('Importance vs Correlation')

# Add feature names to points
for i, feature in enumerate(feature_importance_df['Feature']):
    plt.annotate(feature, (feature_importance_df.iloc[i]['Importance'], correlation_values[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=8)

# Cumulative importance
plt.subplot(2, 2, 4)
cumulative_importance = np.cumsum(feature_importance_df['Importance'])
plt.plot(range(1, len(cumulative_importance) + 1), cumulative_importance, 'bo-')
plt.axhline(y=0.8, color='r', linestyle='--', label='80% threshold')
plt.xlabel('Number of Features')
plt.ylabel('Cumulative Importance')
plt.title('Cumulative Feature Importance')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 4. Feature insights
print(f"\nKey Insights:")
print(f"• Most important feature: {feature_importance_df.iloc[0]['Feature']} ({feature_importance_df.iloc[0]['Importance_Percentage']:.1f}%)")
print(f"• Top 3 features account for {feature_importance_df.head(3)['Importance_Percentage'].sum():.1f}% of importance")

# Find how many features are needed for 80% importance
cumsum = np.cumsum(feature_importance_df['Importance'])
features_for_80_percent = np.where(cumsum >= 0.8)[0][0] + 1
print(f"• {features_for_80_percent} features needed to explain 80% of model decisions")

# 5. Feature descriptions based on importance
feature_descriptions = {
    'LSTAT': 'Lower status population percentage - strong negative correlation with price',
    'RM': 'Average number of rooms - more rooms typically mean higher prices',
    'NOX': 'Nitric oxide concentration - pollution indicator affecting desirability',
    'DIS': 'Distance to employment centers - accessibility factor',
    'CRIM': 'Crime rate - safety factor affecting property values',
    'AGE': 'Age of buildings - newer buildings often more valuable',
    'TAX': 'Property tax rate - affects affordability and attractiveness',
    'PTRATIO': 'Pupil-teacher ratio - education quality indicator',
    'CHAS': 'Charles River proximity - waterfront premium',
    'INDUS': 'Industrial area proportion - affects desirability',
    'ZN': 'Large lot zoning - affects neighborhood character',
    'RAD': 'Highway accessibility - transportation convenience',
    'B': 'Racial composition statistic'
}

print(f"\nTop Features Explained:")
for i, feature in enumerate(feature_importance_df.head(5)['Feature']):
    description = feature_descriptions.get(feature, 'No description available')
    print(f"{i+1}. {feature}: {description}")

## 14. Make Predictions on New Data

Finally, let's demonstrate how to use our trained model to make predictions on new, unseen data.

In [ ]:
# Create some hypothetical new houses for prediction
# Let's create realistic scenarios based on our feature understanding

new_houses = pd.DataFrame({
    'CRIM': [0.1, 5.0, 0.05],           # Low, high, very low crime rate
    'ZN': [20.0, 0.0, 50.0],            # Various zoning
    'INDUS': [5.0, 15.0, 2.0],          # Industrial proportion
    'CHAS': [1, 0, 1],                  # Charles River: Yes, No, Yes
    'NOX': [0.4, 0.7, 0.3],             # Low, high, very low pollution
    'RM': [7.5, 5.0, 8.2],              # 7.5, 5, 8.2 rooms
    'AGE': [30.0, 80.0, 10.0],          # Building age
    'DIS': [4.0, 2.0, 6.0],             # Distance to employment
    'RAD': [3, 24, 2],                  # Highway access
    'TAX': [300, 600, 250],             # Property tax
    'PTRATIO': [15.0, 20.0, 12.0],      # Pupil-teacher ratio
    'B': [390.0, 300.0, 395.0],         # Racial composition
    'LSTAT': [5.0, 25.0, 3.0]           # Lower status %
})

# Create the engineered features for new houses
new_houses['ROOMS_PER_CAPITA'] = new_houses['RM'] / (new_houses['CRIM'] + 1)
new_houses['TAX_PER_ROOM'] = new_houses['TAX'] / new_houses['RM']
new_houses['DIS_RAD_INTERACTION'] = new_houses['DIS'] * new_houses['RAD']

# Select the same features used in training
new_houses_selected = new_houses[top_features]

print("New Houses Data:")
print("=" * 60)
print(new_houses_selected)

# Make predictions using our best model
predictions = best_rf_model.predict(new_houses_selected)

# Create a results summary
house_descriptions = [
    "Luxury House: Low crime, waterfront, many rooms, low pollution",
    "Budget House: High crime, no waterfront, fewer rooms, high pollution", 
    "Premium House: Very low crime, waterfront, many rooms, excellent area"
]

results_df = pd.DataFrame({
    'House Description': house_descriptions,
    'Predicted Price ($1000s)': predictions,
    'Predicted Price ($)': predictions * 1000
})

print(f"\nPrediction Results:")
print("=" * 60)
for idx, row in results_df.iterrows():
    print(f"\n{idx + 1}. {row['House Description']}")
    print(f"   Predicted Price: ${row['Predicted Price ($)']:,.0f}")
    print(f"   (${row['Predicted Price ($1000s)']:.1f}k)")

# Let's also create a prediction confidence analysis
# Make multiple predictions with different random states to estimate uncertainty
n_predictions = 100
uncertainty_predictions = []

for i in range(n_predictions):
    # Create a new model with different random state
    temp_model = RandomForestRegressor(**best_rf_model.get_params())
    temp_model.set_params(random_state=i)
    temp_model.fit(X_train, y_train)
    temp_predictions = temp_model.predict(new_houses_selected)
    uncertainty_predictions.append(temp_predictions)

uncertainty_predictions = np.array(uncertainty_predictions)

# Calculate prediction intervals
prediction_mean = uncertainty_predictions.mean(axis=0)
prediction_std = uncertainty_predictions.std(axis=0)
lower_bound = prediction_mean - 1.96 * prediction_std
upper_bound = prediction_mean + 1.96 * prediction_std

print(f"\nPrediction Confidence Intervals (95%):")
print("=" * 60)
for i, description in enumerate(house_descriptions):
    print(f"\n{i + 1}. {description}")
    print(f"   Mean Prediction: ${prediction_mean[i] * 1000:,.0f}")
    print(f"   95% Confidence Interval: ${lower_bound[i] * 1000:,.0f} - ${upper_bound[i] * 1000:,.0f}")
    print(f"   Uncertainty: ±${prediction_std[i] * 1000:,.0f}")

# Visualize predictions
plt.figure(figsize=(12, 8))

# Bar plot of predictions
plt.subplot(2, 2, 1)
colors = ['green', 'orange', 'blue']
bars = plt.bar(range(len(predictions)), predictions, color=colors, alpha=0.7)
plt.xlabel('House Number')
plt.ylabel('Predicted Price ($1000s)')
plt.title('House Price Predictions')
plt.xticks(range(len(predictions)), ['House 1\n(Luxury)', 'House 2\n(Budget)', 'House 3\n(Premium)'])

# Add value labels on bars
for bar, pred in zip(bars, predictions):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'${pred:.1f}k', ha='center', va='bottom')

# Feature comparison for the three houses
plt.subplot(2, 2, 2)
important_features = ['LSTAT', 'RM', 'NOX', 'DIS']
x = np.arange(len(important_features))
width = 0.25

for i in range(3):
    values = [new_houses_selected.iloc[i][feature] for feature in important_features]
    plt.bar(x + i*width, values, width, label=f'House {i+1}', color=colors[i], alpha=0.7)

plt.xlabel('Features')
plt.ylabel('Feature Values')
plt.title('Key Features Comparison')
plt.xticks(x + width, important_features)
plt.legend()

# Prediction uncertainty
plt.subplot(2, 2, 3)
plt.errorbar(range(len(prediction_mean)), prediction_mean, yerr=1.96*prediction_std, 
             fmt='o', capsize=5, capthick=2, color='red')
plt.xlabel('House Number')
plt.ylabel('Predicted Price ($1000s)')
plt.title('Predictions with Confidence Intervals')
plt.xticks(range(len(predictions)), ['House 1', 'House 2', 'House 3'])

# Feature importance for new predictions
plt.subplot(2, 2, 4)
# Calculate how much each feature contributes to the difference from average
avg_price = y_train.mean()
feature_contributions = []

for i in range(len(predictions)):
    contributions = []
    for feature in important_features:
        feature_idx = list(new_houses_selected.columns).index(feature)
        # Simplified contribution calculation (this is approximate)
        feature_importance = best_rf_model.feature_importances_[feature_idx]
        feature_value = new_houses_selected.iloc[i][feature]
        feature_mean = X_train[feature].mean()
        contribution = feature_importance * (feature_value - feature_mean) / feature_mean
        contributions.append(contribution)
    feature_contributions.append(contributions)

# Plot contributions for House 1 (example)
plt.barh(important_features, feature_contributions[0], color='green', alpha=0.7)
plt.xlabel('Relative Feature Contribution')
plt.title('Feature Contributions (House 1)')

plt.tight_layout()
plt.show()

print(f"\n🎯 Summary:")
print(f"Our machine learning model successfully predicts house prices!")
print(f"Key factors: Low crime, more rooms, less pollution, better location")
print(f"Price range: ${min(predictions)*1000:,.0f} - ${max(predictions)*1000:,.0f}")

## 🎉 Congratulations! You've Completed the Machine Learning Tutorial

### What You've Learned:

1. **Data Loading and Exploration**: How to load datasets and understand their structure
2. **Data Preprocessing**: Cleaning data, handling missing values, and feature engineering
3. **Exploratory Data Analysis**: Visualizing data patterns and relationships
4. **Feature Selection**: Identifying the most important features for modeling
5. **Model Training**: Implementing Linear Regression, Decision Trees, and Random Forest
6. **Model Evaluation**: Using metrics like MSE, RMSE, and R² to assess performance
7. **Cross-Validation**: Getting robust performance estimates
8. **Hyperparameter Tuning**: Optimizing model parameters for better performance
9. **Feature Importance**: Understanding which features drive predictions
10. **Making Predictions**: Using trained models on new, unseen data

### Key Takeaways:

- **Random Forest** often performs better than single models due to ensemble learning
- **Feature engineering** can significantly improve model performance
- **Cross-validation** provides more reliable performance estimates than single train-test splits
- **Hyperparameter tuning** can squeeze out extra performance from your models
- **Feature importance** helps understand and trust your model's decisions

### Next Steps:

- Try different datasets and problem types (classification vs regression)
- Explore more advanced algorithms (XGBoost, Neural Networks, SVM)
- Learn about feature selection techniques (RFE, SelectKBest)
- Study model interpretation techniques (SHAP, LIME)
- Practice with real-world datasets from Kaggle or UCI ML Repository

### Resources for Further Learning:

- **Scikit-learn Documentation**: Comprehensive guide to ML in Python
- **Kaggle Learn**: Free micro-courses on machine learning topics
- **Coursera/edX**: University-level ML courses
- **Hands-On Machine Learning** by Aurélien Géron: Excellent ML book
- **Python Machine Learning** by Sebastian Raschka: Deep dive into ML concepts

Happy machine learning! 🚀